# svm classifier: using tfidf vectorizer

-----------------------------




## Importar librerías y utiles

In [1]:
import pandas as pd
import shutil

#tokenizadores
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#clasificadores
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
#pipeline
from sklearn.pipeline import Pipeline
#metricas
from sklearn.metrics import make_scorer, confusion_matrix, cohen_kappa_score, classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.util import mark_negation
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.util import mark_negation
from sklearn.model_selection import GridSearchCV
#limpieza
from nltk.corpus import stopwords
import string
import nltk

#graficos
import matplotlib.pyplot as plt

import os
import numpy as np

## Datos

### Obtener los datasets desde el github del curso

In [2]:
train = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/anger-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/fear-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/joy-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/sadness-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'])
}

target = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/anger-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/fear-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/joy-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/sadness-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE'])
}

data_keys = ['anger','fear','joy','sadness']

### Analizar los datos 

Imprimir la cantidad de tweets de cada dataset, según su intensidad de sentimiento

In [3]:
def get_group_dist(group_name, train):
    print(group_name, "\n",
          train[group_name].groupby('sentiment_intensity').count())


for key in train:
    get_group_dist(key, train)

anger 
                       id  tweet  class
sentiment_intensity                   
high                 163    163    163
low                  161    161    161
medium               617    617    617
fear 
                       id  tweet  class
sentiment_intensity                   
high                 270    270    270
low                  288    288    288
medium               699    699    699
joy 
                       id  tweet  class
sentiment_intensity                   
high                 195    195    195
low                  219    219    219
medium               488    488    488
sadness 
                       id  tweet  class
sentiment_intensity                   
high                 197    197    197
low                  210    210    210
medium               453    453    453


## Clasificar

In [4]:
# Metrica de evaluación. No tocar...
def auc(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set])
    low_predicted = np.array([prediction[0] for prediction in predicted_set])

    high_test = np.where(test_set == 'high', 1.0, 0.0)
    medium_test = np.where(test_set == 'medium', 1.0, 0.0)
    low_test = np.where(test_set == 'low', 1.0, 0.0)

    auc_high = roc_auc_score(high_test, high_predicted)
    auc_med = roc_auc_score(medium_test, medium_predicted)
    auc_low = roc_auc_score(low_test, low_predicted)

    auc_w = (low_test.sum() * auc_low + medium_test.sum() * auc_med +
             high_test.sum() * auc_high) / (
                 low_test.sum() + medium_test.sum() + high_test.sum())
    return auc_w    

### Dividir el dataset en entrenamiento y prueba

In [5]:
def split_dataset(dataset):
    # Dividir el dataset en train set y test set
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.tweet,
        dataset.sentiment_intensity,
        shuffle=True,
        test_size=0.33,
        stratify= dataset.sentiment_intensity)
    return X_train, X_test, y_train, y_test

## Tokenizador manual

combinación de tweet_tokenizer + lemmatizer + mark_negation

In [6]:
def overkill_tokenize(sentence):
    punt = list(string.punctuation)
    punt.remove("!")
    remove_words = stopwords.words('english') + punt
    sentence = sentence.lower()
    # first: tweet tokenize
    tweetTok = TweetTokenizer(strip_handles=True, reduce_len=True).tokenize
    lemma = WordNetLemmatizer().lemmatize
    preprocess_tweet = tweetTok(sentence)
    
    # second: lemmatization
    preprocess_tweetlemma = []
    for word in preprocess_tweet:
        if not word in remove_words:
            preprocess_tweetlemma.append(lemma(word))
    # third: add mark_negation
    preprocess_tweetLemmaMark = mark_negation(preprocess_tweetlemma)
    return preprocess_tweetLemmaMark

### Definir el clasificador

Consejo para el vectorizador: investigar los modulos de `nltk`, en particular, `TweetTokenizer`, `mark_negation`. También, el parámetro ngram_range para clasificadores no bayesianos.

Consejo para el clasificador: investigar otros clasificadores mas efectivos que naive bayes. Ojo q naive bayes no debería usarse con n-gramas, ya que rompe el supuesto de independencia.


In [7]:
# Experimentación
#  strip_handles: elimina el usuario que emite el tweet
#  reduce_len: estandariza largos, si cualquier elemento len(elem)> 3, se dejan 3 de elem
# Ejemplo:
tknzr = TweetTokenizer(strip_handles=False, reduce_len=False)
s1 = '@remy: This is waaaaayyyy toooooo much for you!!!!!!'
tknzr.tokenize(s1)

tknzr = TweetTokenizer(strip_handles=True, reduce_len=False)
s1 = '@remy: This is waaaaayyyy toooooo much for you!!!!!!'
tknzr.tokenize(s1)

tknzr = TweetTokenizer(strip_handles=True, reduce_len=True) # Más útil
s1 = '@remy: This is waaaaayyyy toooooo much for you!!!!!!'
tknzr.tokenize(s1)


[':', 'This', 'is', 'waaayyy', 'tooo', 'much', 'for', 'you', '!', '!', '!']

In [8]:
# Definimos el pipeline con el vectorizador y el clasificador.
def get_classifier(c_value=1.0):
    # Inicializamos el Vectorizador
    vectorizor = TfidfVectorizer(ngram_range = (1,3), stop_words='english')
    
    # Inicializamos el Clasificador: Aquí cambiamos el clasificador por algo especializado en multiples clases
    oneforall = OneVsRestClassifier(SVC(kernel='linear', C= c_value, probability=True, gamma='scale'))
    
    # Establecer el pipeline.
    text_clf = Pipeline([('vect', vectorizor), ('clf', oneforall)])
    return text_clf

### Definir evaluación

Esta función imprime la matriz de confusión, el reporte de clasificación y las metricas usadas en la competencia:


- `auc`
- `kappa`
- `accuracy`

In [9]:
def evaulate(predicted, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador. 
    # (que comunmente, es distinto a ['low', 'medium', 'high'])
    predicted_labels = [labels[np.argmax(item)] for item in predicted]
    
    # Confusion Matrix
    print('Confusion Matrix for {}:\n'.format(key))

    # Classification Report
    print(
        confusion_matrix(y_test,
                         predicted_labels,
                         labels=['low', 'medium', 'high']))

    print('\nClassification Report')
    print(
        classification_report(y_test,
                              predicted_labels,
                              labels=['low', 'medium', 'high']))

    # AUC
    print("auc: ", auc(y_test, predicted))

    # Kappa
    print("kappa:", cohen_kappa_score(y_test, predicted_labels))

    # Accuracy
    print("accuracy:", accuracy_score(y_test, predicted_labels), "\n")

    print('------------------------------------------------------\n\n')

## cross validation 

generar cross validation para manejar estadisticas

In [10]:
def statisticsCV(clf, dataset, n_cv=10, eval_metric='accuracy'):
    metrics = cross_val_score(clf, dataset.tweet, dataset.sentiment_intensity, cv= n_cv, scoring=eval_metric)
    mean = np.mean(metrics)
    std = np.std(metrics)
    return mean, std, eval_metric

### Ejecutar el clasificador para cierto dataset

Clasifica un dataset. Retorna el modelo ya entrenado mas sus labels asociadas.


In [11]:
def classify(dataset, key):

    X_train, X_test, y_train, y_test = split_dataset(dataset)
    text_clf = get_classifier()

    # Entrenar el clasificador
    #mark_negation(X_train, shallow=True)
    text_clf.fit(X_train, y_train)

    # Predecir las probabilidades de intensidad de cada elemento del set de prueba.
    #mark_negation(X_test, shallow=True)
    predicted = text_clf.predict_proba(X_test)
    # Obtener las clases aprendidas.
    learned_labels = text_clf.classes_

    # Evaluar
    evaulate(predicted, y_test, learned_labels)
    
    # Estadisticas CV
    mean, std, eval_metric = statisticsCV(text_clf, dataset, n_cv=10)
    print('{} for {}:\n'.format(eval_metric, key))
    print("mean: ", mean, "|  std: ", std)
    print('\n')
    
    
    # Estadisticas CV
    mean, std, eval_metric = statisticsCV(text_clf, dataset, n_cv=10, eval_metric=make_scorer(cohen_kappa_score))
    print('{} for {}:\n'.format(eval_metric, key))
    print("mean: ", mean, "|  std: ", std)
    print('\n')
    
    print("--------------------------------------------------------------")
    
    return text_clf, learned_labels

### Ejecutar el clasificador por cada dataset


In [12]:
classifiers = []
learned_labels_array = []

# Por cada llave en train ('anger', 'fear', 'joy', 'sadness')
for key in train:
    classifier, learned_labels = classify(train[key], key)
    classifiers.append(classifier)
    learned_labels_array.append(learned_labels)

Confusion Matrix for anger:

[[  7  46   0]
 [  5 182  17]
 [  0  35  19]]

Classification Report
              precision    recall  f1-score   support

         low       0.58      0.13      0.22        53
      medium       0.69      0.89      0.78       204
        high       0.53      0.35      0.42        54

    accuracy                           0.67       311
   macro avg       0.60      0.46      0.47       311
weighted avg       0.64      0.67      0.62       311

auc:  0.470506189756227
kappa: 0.2088468472918571
accuracy: 0.6688102893890675 

------------------------------------------------------


accuracy for anger:

mean:  0.6633074600537033 |  std:  0.03598431608208032


make_scorer(cohen_kappa_score) for anger:

mean:  0.20661881829564033 |  std:  0.05806118450930706


--------------------------------------------------------------
Confusion Matrix for fear:

[[ 24  68   3]
 [ 21 178  32]
 [  5  39  45]]

Classification Report
              precision    recall  f1-score 

## Predecir target set

In [13]:
def predict_target(dataset, classifier, labels):
    # Predecir las probabilidades de intensidad de cada elemento del target set.
    predicted = pd.DataFrame(classifier.predict_proba(dataset.tweet), columns=labels)
    # Agregar ids
    predicted['id'] = dataset.id.values
    # Reordenar
    predicted = predicted[['id', 'low', 'medium', 'high']]
    return predicted

### Ejecutar la predicción y guardar archivos.

In [14]:
predicted_target = {}

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

for idx, key in enumerate(target):
    # Predecir el target set
    predicted_target[key] = predict_target(target[key], classifiers[idx],
                                           learned_labels_array[idx])
    # Guardar predicciones
    predicted_target[key].to_csv('./predictions/{}-pred.txt'.format(key),
                                 sep='\t',
                                 header=False,
                                 index=False)

# Crear archivo zip
a = shutil.make_archive('predictions', 'zip', './predictions')